![HPEDEVlogo](Pictures/hpedevlogo-NB.JPG)    ![Dockerlogo](Pictures/docker.png)  

Let's start by connecting to our docker appliance (Docker in Docker Appliance)

In [ ]:
%login {{ hostvars[inventory_hostname]['IP-WKSHP-Docker101'] }}

In [ ]:
%login 16.31.86.200

# Using Docker
Estimated time: 15 minutes.
## The first container
In order to be able to manage a first container, the easiest approach is to import an existing one, before creating your own. For that we will refer to the public Docker registry which contains thousands of ready to be consumed containers, including this hello-world one:

In [ ]:
docker run hello-world

So we've got a success! Of course, we do not really go far first, but what can you expect from an hello-world example ;-). 
Let's understand what happened by getting informations on images and containers created with the `ls` command:

In [ ]:
docker image ls

In [ ]:
docker container ls -a

So we see that we now have an image which has been downloaded from the Docker public registry (because it mentionned theimage wasn't found locally and was pulled), and that a container has been instantiated from that image and is not running anymore (Status Exited). The `rm` command allows to delete the container (but of course not the image which remains available)

In [ ]:
docker container ls -a | grep hello-world | awk '{print $1}'| xargs docker container rm

If everything goes well, it should have printed the Id of the container removed. Check that:

In [ ]:
docker container prune -f

In [ ]:
docker container ls -a

## The second container
In order to have a more interesting environment, we'll now look for existing container images in the public Docker registry, and choose to use an ubuntu docker image on our host environment:

In [ ]:
docker search ubuntu

In [ ]:
docker pull ubuntu

Once the container image has been downloaded we can view it in our catalog of images:

In [ ]:
docker image ls

This content is called an image and will serve as the base to create the operational container (here based on Ubuntu) in which we will process data:

In [ ]:
docker run ubuntu:latest cat /etc/debian_version

whereas on our host:

In [ ]:
cat /etc/debian_version

This example shows that you can run completely different environments inside a container as our local Linux distribution is not based on Debian at all, contrary to Ubuntu.
Let's try to improve our container by installing wget in it:

In [ ]:
docker run ubuntu:latest bash -c "apt update && apt-get install -y wget"

So you have successfully install the `wget` command in your Ubuntu container. Hurray !
Let's try to use it:

In [ ]:
docker run ubuntu:latest wget http://www.google.com

Now you should start feeling depressed :-) Why is it not working whereas you have seen it was installed successfully just upper !!
Well, we need to understand what really a container is and the nature of the commands we passed ealier for that. Let's examine containers:

In [ ]:
docker container ls -a

You should see at least 3 Ubuntu based containers with the different commands we passed. Because each time we run `docker run`, Docker instantiates a **new** container and doesn't reuse any of the previous created. So we now have 3 different execution environments all based on Ubuntu but all with their own lifecycle. Meaning that in the last one, the installation of `wget` never happened so the command is not found. It is in the second container only. We'll see how to deal with that later on.

Let's understand another aspect:

In [ ]:
docker run ubuntu:latest uname -a

Because we really are on a CentOS distribution, what you see is the unique kernel running all these tasks, whether they are in a container or native. Anyway all run using the same kernel (and kernel APIs) which is shared across all containers and native processes (that containers encapsulate). 
Now let's try to interact with the Linux package manager:

In [ ]:
 docker run ubuntu dpkg -l

Which, as you can check, doesn't contain the `wget` command and of course doesn't work locally as CentOS is rpm based:

In [ ]:
dpkg -l

In [ ]:
rpm -aq | wc -l

You see that the underlying Linux distribution doesn't understand the `dpkg` command. It is only valid in the container environment, which is an Ubuntu distribution whereas our base distribution is a CentOS one, with another package management. The last command gives you the number of packages installed.

However, we can get some info on our modified Docker environment:

In [ ]:
docker image ls

So you see that you now have 2 new images from which our 2 test containers have been created (the hello-world and the Ubuntu)

In [ ]:
docker container ls -a

And here you see the list of your containers with their status (they should all be Exited as they have finished their task, except the one calling `wget` which failed)

So to go back to our previous question, why isn't the `wget` command available to retrieve a Web page ? Well this is because of the evanescent aspect of the container. Each time you made a command in one, it's only valid for that one, as it has its own life cycle. So the `wget` command indeed exists in the container in which you installed it, but not in all the orthers where you passed different commands. Check that by reusing the container where you installed `wget`

In [ ]:
wgetid=`docker container ls -a | grep 'apt update' | awk {'print $1}'`
docker diff $wgetid | grep wget


So you can verify that the `wget` command has indeed been well installed. If you run the request with the install, this will probably be a better proof:

In [ ]:
docker run ubuntu:latest bash -c "apt update && apt-get install -y wget && wget http://www.hpe.com"

Now that you start getting it, let's clean some stuff before going on. In order to remove the ubuntu container(s), we first need to retrieve their container id (first column with awk) and to use the docker rm command to remove them:

In [ ]:
docker container ls -a | grep ubuntu | awk '{ print $1}' | xargs docker container rm 

Check they are gone:

In [ ]:
docker container ls -a

Let's now work on something more useful.

<br><br>

## <i class="fas fa-2x fa-map-marker-alt" style="color:#551199;"></i>&nbsp;&nbsp;Next Steps

# Lab2 : Using Dockerfile

<h2>Next LAB&nbsp;&nbsp;&nbsp;&nbsp;<a href="3-WKSHP-Using-Dockerfile.ipynb" target="New" title="Next LAB: Using Dockerfile"><i class="fas fa-chevron-circle-right" style="color:#551199;"></i></a></h2>

</br>
 <a href="1-WKSHP-Intro-to-Containers-techno.ipynb" target="New" title="Back: Introduction to Containers technologies"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#551199;color:#fff;position:relative;width:10%; height: 30px;float: left;"><b>Back</b></button></a>
 <a href="3-WKSHP-Using-Dockerfile.ipynb" target="New" title="Next:Using Dockerfile"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#551199;color:#fff;position:relative;width:10%; height: 30px;float: right;"><b>Next</b></button></a>
